In [1]:
from __future__ import division
import pandas as pd
from urbansim.utils import misc

### Steps to get baseline results

* Get block ids in household and jobs tables via parcels table
* Save to new HDF5 file and set up config file
* Set up indicators container on Paris
* Set up notebook to get results from indicators API on Paris

In [6]:
# Read parcel-to-block table
parcel_block = pd.read_csv('parcel_to_block.csv', dtype=str)
parcel_block = parcel_block.set_index('parcel_id')

# Read block geometry table and set block_id to index of blocks table
block_geom = pd.read_csv('block_geoms.csv', dtype='str')
blocks = pd.DataFrame(index=block_geom.block_id)

In [7]:
len(blocks)

109228

In [8]:
# Read HDF5 file of baseline data
with pd.HDFStore('./../data/2015_09_01_bayarea_v3.h5') as store:
    parcels = store.parcels
    buildings = store.buildings
    households = store.households
    jobs = store.jobs

In [9]:
# Read existing file

store = pd.HDFStore('/home/paul/Documents/Projects/indicators/data/bay_area_block_model_data.h5')

In [10]:
# Get block info to blocks table
blocks['square_meters_land'] = store.blocks.square_meters_land

In [11]:
# Get block_ids to parcels and buildings
parcels['block_id'] = misc.reindex(parcel_block.block_id, parcels.index.astype(str))
buildings['block_id'] = misc.reindex(parcel_block.block_id, buildings.parcel_id.astype(str))

In [12]:
# Get block_ids to jobs and households

jobs['block_id'] = misc.reindex(buildings.block_id, jobs.building_id)
households['block_id'] = misc.reindex(buildings.block_id, households.building_id)

In [13]:
new_store = pd.HDFStore('./../data/bayarea_block_baseline.h5', mode='w')

In [14]:
new_store['parcels'] = parcels
new_store['buildings'] = buildings
new_store['blocks'] = blocks
new_store['households'] = households
new_store['jobs'] = jobs

/home/paul/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['apn', 'parcel_id_local', 'imputation_flag', 'block_id']]

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/paul/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['redfin_home_type', 'costar_property_type', 'costar_rent', 'block_id']]

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/paul/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,k

In [15]:
new_store['drive_edges'] = store.drive_edges
new_store['drive_nodes'] = store.drive_nodes

In [16]:
new_store.close()
store.close()

In [2]:
new_store = pd.HDFStore('./../data/bayarea_block_baseline.h5', mode='r')

In [4]:
new_store.buildings.dtypes

parcel_id                 int64
development_type_id       int64
improvement_value       float64
residential_units         int64
residential_sqft          int64
sqft_per_unit           float64
non_residential_sqft      int64
building_sqft           float64
nonres_rent_per_sqft    float64
res_price_per_sqft      float64
stories                   int64
year_built                int64
redfin_sale_price       float64
redfin_sale_year        float64
redfin_home_type         object
costar_property_type     object
costar_rent              object
building_type_id          int64
block_id                 object
dtype: object